# Добавление признаков эстетичности

## Imports

In [6]:
import numpy as np
import pandas as pd
from skimage import color
from skimage.filters.rank import entropy
from skimage.morphology import disk
from PIL import Image
import swifter

In [12]:
import pandas as pd
import requests

from tqdm import tqdm
tqdm.pandas()

from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 10 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


## Definitions
Определим функции, которые вычисляют метрики эстетичности

In [19]:
def CFN(img):
    pxls = color.rgb2lab(img[:, :, :3]).reshape([img.shape[0] * img.shape[1], 3])

    stda = np.std(pxls[:, 1])
    stdb = np.std(pxls[:, 2])

    centre = (
        np.sum(pxls[:, 1]) / pxls.shape[0],
        np.sum(pxls[:, 2]) / pxls.shape[0]
    )

    return int(np.hypot(stda, stdb) + 0.37 * np.hypot(centre[0], centre[1]))


def img_entropy(img):
    try:
        img = color.rgb2gray(img)
    except:
        img = color.rgba2gray(img)

    return np.mean(entropy(img, disk(5)))


def rgb_box_count(img, box_size=2):
    R, G, B = (img[:, :, i].flatten() for i in range(3))

    boxes = np.full((256 // box_size, 256 // box_size, 256 // box_size), False)

    for (r, g, b) in np.array([R, G, B]).T:
        boxes[
            int(r // box_size),
            int(g // box_size),
            int(b // box_size)
        ] = True

    return - np.log(np.sum(boxes)) / np.log(box_size / 256)



## Главная функция
Эту функцию мы применим ко всем строкам нашего датасета, и получим на выход 3 метрики эстетичности, которые будут добавлены к датасету

In [20]:
from pathlib import Path

def add_aesthetics_metrics(row, src=Path('./../images')):
    # TODO: поменяейте здесь ./src на путь до папки, где хранятся ваши картинки
    img_path = src / row['image']
    img = np.array(Image.open(img_path))

    cfn = CFN(img)
    entropy = img_entropy(img)
    # box_count = rgb_box_count(img)
    box_count = 0

    return pd.Series([cfn, entropy, box_count], index=['cfn', 'entropy', 'box_count'])

In [21]:
from tqdm import tqdm
tqdm.pandas()

dataset_path = './christies_auction_clean_embeddings.csv'

# Read the dataset we have
df = pd.read_csv(dataset_path)
print('Given dataset has shape of:', df.shape)

Given dataset has shape of: (8832, 2104)


In [22]:
# Compute aesthetics metrics
df[['cfn', 'entropy', 'box_count']] = df.parallel_apply(add_aesthetics_metrics, axis=1)
print('After processing dataset has shape of:', df.shape)

# Store results into .csv for Orange use
df.to_csv('./christies_auction_clean_embeddings_aesthetics.csv', index=True)

/var/folders/35/tb4whj3x0t18xb6yn70mvqs8cq9clk/T/ipykernel_10206/553822965.py:9: UserWarning: Possible precision loss converting image of type float64 to uint8 as required by rank filters. Convert manually using skimage.util.img_as_ubyte to silence this warning.
  entropy = img_entropy(img)
/var/folders/35/tb4whj3x0t18xb6yn70mvqs8cq9clk/T/ipykernel_10206/553822965.py:9: UserWarning: Possible precision loss converting image of type float64 to uint8 as required by rank filters. Convert manually using skimage.util.img_as_ubyte to silence this warning.
  entropy = img_entropy(img)
/var/folders/35/tb4whj3x0t18xb6yn70mvqs8cq9clk/T/ipykernel_10206/553822965.py:9: UserWarning: Possible precision loss converting image of type float64 to uint8 as required by rank filters. Convert manually using skimage.util.img_as_ubyte to silence this warning.
  entropy = img_entropy(img)
/var/folders/35/tb4whj3x0t18xb6yn70mvqs8cq9clk/T/ipykernel_10206/553822965.py:9: UserWarning: Possible precision loss conver

KeyboardInterrupt: 